# Evaluación del Modelo
## Predicción de Clientes Fidelizables en E-commerce

### Objetivos de la Evaluación

Este notebook se enfoca en la evaluación comprehensiva del modelo de fidelización, con especial atención a la prevención del overfitting. Los objetivos específicos incluyen:

1. **Evaluación robusta del rendimiento**
   - Métricas de clasificación en múltiples conjuntos
   - Validación cruzada estratificada
   - Análisis de curvas de aprendizaje
   - Detección de overfitting/underfitting

2. **Análisis de generalización**
   - Comparación train vs validation vs test
   - Estabilidad de métricas across folds
   - Análisis de varianza del modelo

3. **Interpretabilidad del modelo**
   - Importancia de características
   - Análisis de casos límite
   - Explicabilidad de predicciones

### Marco de Evaluación

La evaluación sigue principios rigurosos de validación:
- **Separación estricta** de conjuntos train/validation/test
- **Validación cruzada estratificada** para robustez
- **Múltiples métricas** para evaluación integral
- **Análisis de estabilidad** temporal y estadística

In [19]:
# Configuración inicial
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Librerías de evaluación
from sklearn.metrics import (
    classification_report, confusion_matrix, roc_auc_score, roc_curve,
    precision_recall_curve, average_precision_score, accuracy_score,
    f1_score, precision_score, recall_score
)
from sklearn.model_selection import (
    learning_curve, validation_curve, cross_val_score, 
    StratifiedKFold, cross_validate
)
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest

import joblib
import json
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("Librerías de evaluación cargadas")
print(f"Fecha de evaluación: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Librerías de evaluación cargadas
Fecha de evaluación: 2025-09-01 13:42:38


In [20]:
# Cargar modelo y componentes entrenados
try:
    best_model = joblib.load('../results/models/best_loyalty_model.pkl')
    scaler = joblib.load('../results/models/feature_scaler.pkl')
    selector = joblib.load('../results/models/feature_selector.pkl')
    
    with open('../results/models/model_info.json', 'r') as f:
        model_info = json.load(f)
    
    print(f"Modelo cargado: {model_info['model_name']}")
    print(f"Características seleccionadas: {len(model_info['selected_features'])}")
    
except FileNotFoundError:
    print("Modelos no encontrados. Creando modelo sintético para evaluación...")
    
    # Crear modelo sintético para demostración
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.preprocessing import StandardScaler
    from sklearn.feature_selection import SelectKBest, f_classif
    
    best_model = RandomForestClassifier(n_estimators=100, random_state=42)
    scaler = StandardScaler()
    selector = SelectKBest(score_func=f_classif, k=10)
    
    model_info = {
        'model_name': 'Random Forest',
        'selected_features': ['Recency', 'Frequency', 'Monetary', 'AvgRevenue'],
        'feature_columns': ['Recency', 'Frequency', 'Monetary', 'AvgRevenue']
    }
    
    print("Modelo sintético creado para demostración")

Modelo cargado: Random Forest
Características seleccionadas: 15


## 1. Preparación de Datos para Evaluación

### Estrategia de Validación

Para prevenir overfitting, implementamos una estrategia de validación robusta:
- **División estratificada** manteniendo proporción de clases
- **Validación cruzada** con múltiples folds
- **Conjunto de test independiente** nunca visto durante entrenamiento
- **Métricas de estabilidad** para detectar sobreajuste

In [21]:
# Recrear preparación de datos consistente con entrenamiento
try:
    df = pd.read_csv('../data/processed/customer_features_with_trends.csv')
    print(f"Dataset cargado: {df.shape[0]:,} clientes")
except FileNotFoundError:
    # Crear dataset sintético consistente
    np.random.seed(42)
    n_customers = 1000
    
    df = pd.DataFrame({
        'CustomerID': range(12346, 12346 + n_customers),
        'Recency': np.random.exponential(30, n_customers),
        'Frequency': np.random.poisson(5, n_customers) + 1,
        'Monetary': np.random.gamma(2, 100, n_customers),
        'AvgRevenue': np.random.gamma(2, 50, n_customers),
        'UniqueProducts': np.random.poisson(8, n_customers) + 2,
        'Country': np.random.choice(['UK', 'FR', 'DE'], n_customers, p=[0.7, 0.15, 0.15])
    })
    
    # Variable objetivo con criterios realistas
    df['IsLoyal'] = (
        (df['Frequency'] >= df['Frequency'].quantile(0.6)) & 
        (df['Monetary'] >= df['Monetary'].quantile(0.4)) & 
        (df['Recency'] <= df['Recency'].quantile(0.6))
    ).astype(int)
    
    print(f"Dataset sintético creado: {df.shape[0]:,} clientes")

# Preparar características
feature_columns = model_info.get('feature_columns', ['Recency', 'Frequency', 'Monetary', 'AvgRevenue'])
available_features = [col for col in feature_columns if col in df.columns]

X = df[available_features].fillna(0)
y = df['IsLoyal']

print(f"Características disponibles: {len(available_features)}")
print(f"Distribución de clases: {y.value_counts().to_dict()}")
print(f"Balance de clases: {y.mean():.1%} fidelizables")

Dataset cargado: 4,338 clientes
Características disponibles: 18
Distribución de clases: {0: 2494, 1: 1844}
Balance de clases: 42.5% fidelizables


In [22]:
# División estratificada de datos con validación robusta
from sklearn.model_selection import train_test_split

# División inicial train/test (80/20)
X_train_full, X_test, y_train_full, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# División train/validation (64/16 del total)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_full, y_train_full, test_size=0.2, random_state=42, stratify=y_train_full
)

print("DIVISIÓN DE DATOS PARA EVALUACIÓN ROBUSTA")
print("=" * 45)
print(f"Entrenamiento: {X_train.shape[0]:,} muestras ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"Validación: {X_val.shape[0]:,} muestras ({X_val.shape[0]/len(X)*100:.1f}%)")
print(f"Test: {X_test.shape[0]:,} muestras ({X_test.shape[0]/len(X)*100:.1f}%)")

# Verificar balance en cada conjunto
print(f"\nBalance de clases por conjunto:")
print(f"Train: {y_train.mean():.1%} fidelizables")
print(f"Validation: {y_val.mean():.1%} fidelizables")
print(f"Test: {y_test.mean():.1%} fidelizables")

# Aplicar transformaciones
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Selección de características
X_train_selected = selector.fit_transform(X_train_scaled, y_train)
X_val_selected = selector.transform(X_val_scaled)
X_test_selected = selector.transform(X_test_scaled)

print(f"\nCaracterísticas después de selección: {X_train_selected.shape[1]}")

DIVISIÓN DE DATOS PARA EVALUACIÓN ROBUSTA
Entrenamiento: 2,776 muestras (64.0%)
Validación: 694 muestras (16.0%)
Test: 868 muestras (20.0%)

Balance de clases por conjunto:
Train: 42.5% fidelizables
Validation: 42.5% fidelizables
Test: 42.5% fidelizables

Características después de selección: 15


## 2. Análisis de Curvas de Aprendizaje

### Detección de Overfitting

Las curvas de aprendizaje son fundamentales para detectar:
- **Overfitting**: Gran diferencia entre train y validation
- **Underfitting**: Ambas curvas convergen a un valor bajo
- **Punto óptimo**: Mejor balance bias-variance

In [23]:
# Entrenar modelo si no está entrenado
if not hasattr(best_model, 'feature_importances_') and hasattr(best_model, 'fit'):
    print("Entrenando modelo para evaluación...")
    best_model.fit(X_train_selected, y_train)

# Generar curvas de aprendizaje para detectar overfitting
def plot_learning_curves(model, X, y, cv=5):
    """
    Genera curvas de aprendizaje para detectar overfitting
    """
    train_sizes = np.linspace(0.1, 1.0, 10)
    
    train_sizes_abs, train_scores, val_scores = learning_curve(
        model, X, y, 
        train_sizes=train_sizes,
        cv=StratifiedKFold(n_splits=cv, shuffle=True, random_state=42),
        scoring='roc_auc',
        n_jobs=-1,
        random_state=42
    )
    
    # Calcular estadísticas
    train_mean = np.mean(train_scores, axis=1)
    train_std = np.std(train_scores, axis=1)
    val_mean = np.mean(val_scores, axis=1)
    val_std = np.std(val_scores, axis=1)
    
    return train_sizes_abs, train_mean, train_std, val_mean, val_std

print("ANÁLISIS DE CURVAS DE APRENDIZAJE")
print("=" * 40)

# Generar curvas de aprendizaje
train_sizes, train_mean, train_std, val_mean, val_std = plot_learning_curves(
    best_model, X_train_selected, y_train, cv=5
)

# Visualización con Plotly
fig = go.Figure()

# Curva de entrenamiento
fig.add_trace(go.Scatter(
    x=train_sizes,
    y=train_mean,
    mode='lines+markers',
    name='Training Score',
    line=dict(color='blue'),
    error_y=dict(type='data', array=train_std, visible=True)
))

# Curva de validación
fig.add_trace(go.Scatter(
    x=train_sizes,
    y=val_mean,
    mode='lines+markers',
    name='Validation Score',
    line=dict(color='red'),
    error_y=dict(type='data', array=val_std, visible=True)
))

fig.update_layout(
    title='Curvas de Aprendizaje - Detección de Overfitting',
    xaxis_title='Tamaño del Conjunto de Entrenamiento',
    yaxis_title='ROC-AUC Score',
    height=500
)

fig.show()

# Análisis de overfitting
final_gap = train_mean[-1] - val_mean[-1]
print(f"\nAnálisis de Overfitting:")
print(f"Score final de entrenamiento: {train_mean[-1]:.4f} ± {train_std[-1]:.4f}")
print(f"Score final de validación: {val_mean[-1]:.4f} ± {val_std[-1]:.4f}")
print(f"Gap train-validation: {final_gap:.4f}")

if final_gap > 0.05:
    print("⚠️  ALERTA: Posible overfitting detectado (gap > 0.05)")
elif final_gap < 0.02:
    print("✅ Modelo bien generalizado (gap < 0.02)")
else:
    print("⚡ Overfitting moderado - monitorear")

ANÁLISIS DE CURVAS DE APRENDIZAJE



Análisis de Overfitting:
Score final de entrenamiento: 1.0000 ± 0.0000
Score final de validación: 1.0000 ± 0.0000
Gap train-validation: 0.0000
✅ Modelo bien generalizado (gap < 0.02)


## 3. Validación Cruzada Robusta

### Evaluación de Estabilidad

La validación cruzada estratificada proporciona una evaluación más robusta del rendimiento del modelo y ayuda a detectar inestabilidad que podría indicar overfitting.

In [24]:
# Validación cruzada comprehensiva
def comprehensive_cross_validation(model, X, y, cv_folds=5):
    """
    Realiza validación cruzada con múltiples métricas
    """
    cv = StratifiedKFold(n_splits=cv_folds, shuffle=True, random_state=42)
    
    scoring_metrics = {
        'accuracy': 'accuracy',
        'precision': 'precision',
        'recall': 'recall',
        'f1': 'f1',
        'roc_auc': 'roc_auc'
    }
    
    cv_results = cross_validate(
        model, X, y, 
        cv=cv, 
        scoring=scoring_metrics,
        return_train_score=True,
        n_jobs=-1
    )
    
    return cv_results

print("VALIDACIÓN CRUZADA ROBUSTA")
print("=" * 35)

# Ejecutar validación cruzada
cv_results = comprehensive_cross_validation(best_model, X_train_selected, y_train, cv_folds=5)

# Analizar resultados
metrics_analysis = []

for metric in ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']:
    train_scores = cv_results[f'train_{metric}']
    val_scores = cv_results[f'test_{metric}']
    
    analysis = {
        'Métrica': metric.upper(),
        'Train_Mean': train_scores.mean(),
        'Train_Std': train_scores.std(),
        'Val_Mean': val_scores.mean(),
        'Val_Std': val_scores.std(),
        'Gap': train_scores.mean() - val_scores.mean(),
        'Stability': val_scores.std()  # Menor es mejor
    }
    
    metrics_analysis.append(analysis)

# Crear DataFrame para análisis
cv_df = pd.DataFrame(metrics_analysis)

print("Resultados de Validación Cruzada:")
print(cv_df.round(4))

# Visualización de estabilidad
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Gap Train-Validation', 'Estabilidad de Métricas')
)

# Gap train-validation
fig.add_trace(
    go.Bar(x=cv_df['Métrica'], y=cv_df['Gap'], name='Gap Train-Val'),
    row=1, col=1
)

# Estabilidad (desviación estándar)
fig.add_trace(
    go.Bar(x=cv_df['Métrica'], y=cv_df['Stability'], name='Desviación Estándar'),
    row=1, col=2
)

fig.update_layout(height=400, title_text="Análisis de Estabilidad del Modelo")
fig.show()

# Diagnóstico de overfitting
avg_gap = cv_df['Gap'].mean()
max_gap = cv_df['Gap'].max()
avg_stability = cv_df['Stability'].mean()

print(f"\nDiagnóstico de Overfitting:")
print(f"Gap promedio train-validation: {avg_gap:.4f}")
print(f"Gap máximo: {max_gap:.4f}")
print(f"Estabilidad promedio: {avg_stability:.4f}")

# Criterios de diagnóstico
overfitting_indicators = 0
if avg_gap > 0.05:
    overfitting_indicators += 1
    print("⚠️  Gap promedio alto - posible overfitting")
if max_gap > 0.1:
    overfitting_indicators += 1
    print("⚠️  Gap máximo muy alto - overfitting severo")
if avg_stability > 0.05:
    overfitting_indicators += 1
    print("⚠️  Alta variabilidad - modelo inestable")

if overfitting_indicators == 0:
    print("✅ Modelo estable sin indicios de overfitting")
elif overfitting_indicators <= 1:
    print("⚡ Overfitting leve - aceptable")
else:
    print("🚨 Overfitting significativo - requiere regularización")

VALIDACIÓN CRUZADA ROBUSTA
Resultados de Validación Cruzada:
     Métrica  Train_Mean  Train_Std  Val_Mean  Val_Std     Gap  Stability
0   ACCURACY         1.0        0.0    0.9989   0.0014  0.0011     0.0014
1  PRECISION         1.0        0.0    0.9975   0.0034  0.0025     0.0034
2     RECALL         1.0        0.0    1.0000   0.0000  0.0000     0.0000
3         F1         1.0        0.0    0.9987   0.0017  0.0013     0.0017
4    ROC_AUC         1.0        0.0    1.0000   0.0000  0.0000     0.0000



Diagnóstico de Overfitting:
Gap promedio train-validation: 0.0010
Gap máximo: 0.0025
Estabilidad promedio: 0.0013
✅ Modelo estable sin indicios de overfitting
